# Load libraries

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By

import urllib.request
import multiprocessing as mp

import time

# setup selenium scraper

In [2]:
driver = webdriver.Chrome(executable_path="../dependencies/chromedriver")

In [3]:
url = "https://podcastluisteren.nl/pod/Maarten-van-Rossem-De-Podcast"
driver.get(url)

# Find podcast titles and download links

In [4]:
elements = driver.find_elements(By.XPATH, "//h4[@class='mt-1 text-left']")

In [5]:
titles = [element.text for element in elements]
titles = [title.replace("/", "-") for title in titles]

In [6]:
buttons = [element.find_element(By.XPATH, "../div/button") for element in elements]

In [58]:
def find_audio_path(button, audio_element):
    # Start stream of episode
    button.click()
    # Pause the stream, as we only need it loaded
    button.click()
    time.sleep(0.01)
    
    src = audio_element.get_attribute("src")
    return src

In [59]:
audio = driver.find_element(By.XPATH, "//audio")
sources = [find_audio_path(button, audio) for button in buttons]

### Write titles and src's to files

In [33]:
file = 'Sources.txt'
if not os.path.exists(file):
    with open(file, 'w') as f:
        for line in sources:
            f.write(f"{line}\n")
else:
    with open(file, "r") as f:
        sources = f.read().splitlines()

In [36]:
file = 'Titles.txt'
if not os.path.exists(file):
    with open(file, 'w') as f:
        for line in titles:
            f.write(f"{line}\n")
else:
    with open(file, "r") as f:
        titles = f.read().splitlines()

# Download the mp3 files

In [39]:
use_cores = mp.cpu_count()

In [40]:
def download_mp3(source, title):
    path = f"../data/audio/{title}.mp3"
    if os.path.exists(path):
        return
    urllib.request.urlretrieve(source, path)
    time.sleep(2)

In [41]:
pool = mp.Pool(use_cores)
result = pool.starmap(download_mp3, tuple(zip(sources, titles)))